## Self Ask with Search Agent

In [1]:
import os

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGSMITH_PROJECT"] = LANGSMITH_PROJECT
os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

In [4]:
from langchain.utilities import GoogleSerperAPIWrapper
google_search = GoogleSerperAPIWrapper()

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google import genai

/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", convert_system_message_to_human=True)

The self ask with search agent is an agent type which breaks down a complex problem in simple steps, then tries to answer these intermediate problems and then come to the solution of the complex question. 

For example: 
Question: What is the car that the current F-1 driver used to win his most recent title. 

Intermediate questions might be: What is F-1, who is current F-1 champion, Which team does he belong to, What is the car that this team produces, When did he win his recent title, What was the car used then.

Then it will generate the final answer after answering these questions. 

Unlike the ReAct agent who tries to answer the question by using tools or its own Reasoning, Thought and Action loop, if the dataset does not contain information about this, it will simply return not found.

In [7]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [8]:
from langchain import hub 
from langchain.agents import create_self_ask_with_search_agent, AgentExecutor
from langchain_community.tools.tavily_search import TavilyAnswer
from langchain_core.tools import Tool
from langchain.prompts import PromptTemplate

In [9]:
tools = [
    Tool(
        name = "Intermediate Answer",
        func = google_search.run,
        description = "Useful when you need to search something on the internet.",
        verbose = True
    )
]

In [11]:
prompt = hub.pull("hwchase17/self-ask-with-search")

In [12]:
print(prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

In [13]:
agent = create_self_ask_with_search_agent(llm,tools,prompt)

In [14]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,
                               handle_parsing_errors=True,
                               max_iterations=3, return_intermediate_steps=True)

In [17]:
agent_executor.invoke({"input": "What is the car that the current F-1 driver used to win his most recent title?"})



> Entering new AgentExecutor chain...


/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Yes.
Follow up: Who won the most recent F-1 championship?Max Verstappen (bottom right) is the current World Drivers' Champion, having won the championship four times consecutively. Currently, Sebastian Vettel is the youngest world champion in F1 history. He won his first F1 world title in 2010 at the age of 23 years and 134 days with the ... Third on this shortlist of champions are Sebastian Vettel, Alain Prost and Max Verstappen. They all won 4 F1 drivers' titles. Driver, Nat, Team, Pts. 1, Oscar Piastri, AUS, McLaren, 309. 2, Lando Norris, GBR, McLaren, 275. 3, Max Verstappen, NED, Red Bull, 205. 2025 Season ; 1, Oscar PiastriPIA, Australia ; 2, Lando NorrisNOR, United Kingdom ; 3, Max VerstappenVER, Netherlands ; 4, George RussellRUS, United Kingdom ... Winner: Oscar Piastri. Oscar Piastri's hopes of winning a maiden Formula 1 World Championship in only his third campaign took a big boost in ... The FIA Formula One World Championship has been one of the world's premier forms of moto

/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Intermediate answer: Max Verstappen won the most recent F-1 championship.
Follow up: What car did Max Verstappen drive to win his most recent championship?Verstappen achieved multiple race wins in his 2017 and 2018 campaigns, before finishing third in both the 2019 and 2020 World Drivers' Championships under Honda ... Verstappen secured his second consecutive Drivers' Championship driving the RB18 and Red Bull secured their fifth Constructors' Championship, their first since ... Verstappen crowned champion as Russell heads Mercedes 1-2 in Las Vegas ... Max Verstappen became a four-time world champion with an assured drive ... Max Verstappen Drives the New Ford Mustang GTD with Chris Harris | Ford Performance | Red Bull Ford Powertrains Join four-time Formula 1 ... In the hands of Max Verstappen, the RB19 proves all but invincible, with team winning all but one race en route to a second consecutive championship double. Four-time Formula One World Champion Max Verstappen has been rewriti

/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Intermediate answer: Max Verstappen won his most recent F1 championship in the Red Bull Racing RB19.
So the final answer is: Red Bull Racing RB19

> Finished chain.


{'input': 'What is the car that the current F-1 driver used to win his most recent title?',
 'output': 'Red Bull Racing RB19',
 'intermediate_steps': [(AgentAction(tool='Intermediate Answer', tool_input='Who won the most recent F-1 championship?', log='Yes.\nFollow up: Who won the most recent F-1 championship?'),
   "Max Verstappen (bottom right) is the current World Drivers' Champion, having won the championship four times consecutively. Currently, Sebastian Vettel is the youngest world champion in F1 history. He won his first F1 world title in 2010 at the age of 23 years and 134 days with the ... Third on this shortlist of champions are Sebastian Vettel, Alain Prost and Max Verstappen. They all won 4 F1 drivers' titles. Driver, Nat, Team, Pts. 1, Oscar Piastri, AUS, McLaren, 309. 2, Lando Norris, GBR, McLaren, 275. 3, Max Verstappen, NED, Red Bull, 205. 2025 Season ; 1, Oscar PiastriPIA, Australia ; 2, Lando NorrisNOR, United Kingdom ; 3, Max VerstappenVER, Netherlands ; 4, George Ru

In [19]:
agent_executor.invoke({"input": "How old was the founder of McDonald's when he started the company?"})



> Entering new AgentExecutor chain...


/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Yes.
Follow up: Who is considered the founder of the McDonald's corporation?Kroc became the owner of McDonald's Corporation in 1961 and is credited as its founder, due to his influence as a franchise agent and principal role in the ... Ray Kroc, Founder and Senior Chairman of the Board of McDonald's Corporation, dies on January 14. 1990. On January 31, the first McDonald's restaurant in ... McDonald's is one of the world's largest fast-food chains. Founded by Ray Kroc, it's known for hamburgers, Big Macs, Egg McMuffins, ... Ray Kroc, founder and chairman of McDonald's Corporation, stands outside one of his franchises, holding a hamburger and a drink. Bettman ... Ray Kroc has been called "fast food's founding father." He was born in 1902 and grew up to sell restaurant products including paper cups and milk-shake mixers. In 1955, Ray founded a new corporation called McDonald's Systems, Inc. (predecessor of the McDonald's Corporation) in Chicago, Ill. In 1956, one of the first ... Ray Kro

/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Intermediate answer: Ray Kroc is considered the founder of the McDonald's corporation.
Follow up: When was Ray Kroc born?Ray Kroc: American businessman. Raymond Albert Kroc was an American businessman who was instrumental in turning McDonald's into the most successful global fast food corporation by revenue. He purchased it from the McDonald Brothers in 1961, after several years as their... Ray Kroc Born: October 5, 1902, Oak Park, IL. Ray Kroc Died: January 14, 1984 (age 81 years), San Diego, CA. Ray Kroc Spouse: Joan Kroc (m. 1969–1984), Jane Dobbins Green (m. 1963–1968), and Ethel Fleming (m. 1922–1961). Ray Kroc Children: Marilyn Kroc Barg. Ray Kroc Organizations founded: McDonald's, Hamburger University, and Ronald McDonald House Charities. Ray Kroc Education: Oak Park and River Forest High School. Raymond Albert Kroc was born on October 5, 1902 , in Oak Park, Illinois, near Chicago, to Czech-American parents, Rose Mary [née Hrach] (1881–1959) and Alois "Louis" Kroc (1879–1937). b

/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Intermediate answer: Ray Kroc was born on October 5, 1902.
Follow up: When did Ray Kroc found the McDonald's corporation?Kroc became the owner of McDonald's Corporation in 1961 and is credited as its founder, due to his influence as a franchise agent and principal role in the ... Ray Kroc's first McDonald's restaurant in Des Plaines, Illinois just prior to opening on April 15, 1955. · Newspaper advertisement announcing the opening of Ray ... In April 1955, Kroc launched McDonald's Systems, Inc., later known as McDonald's Corporation, in Des Plaines, Illinois, where he also opened ... In 1955, Kroc founded McDonald's System, Inc., which would later become the McDonald's Corporation. ... company was worth $8 billion. The present corporation credits its founding to franchised businessman Ray Kroc on April 15, 1955. This was the ninth opened McDonald's restaurant ... 1955. Kroc opens his first McDonald's in Des Plaines, Illinois on April 15. The attention getting red and white tiled buildi

{'input': "How old was the founder of McDonald's when he started the company?",
 'output': 'Agent stopped due to iteration limit or time limit.',
 'intermediate_steps': [(AgentAction(tool='Intermediate Answer', tool_input="Who is considered the founder of the McDonald's corporation?", log="Yes.\nFollow up: Who is considered the founder of the McDonald's corporation?"),
   'Kroc became the owner of McDonald\'s Corporation in 1961 and is credited as its founder, due to his influence as a franchise agent and principal role in the ... Ray Kroc, Founder and Senior Chairman of the Board of McDonald\'s Corporation, dies on January 14. 1990. On January 31, the first McDonald\'s restaurant in ... McDonald\'s is one of the world\'s largest fast-food chains. Founded by Ray Kroc, it\'s known for hamburgers, Big Macs, Egg McMuffins, ... Ray Kroc, founder and chairman of McDonald\'s Corporation, stands outside one of his franchises, holding a hamburger and a drink. Bettman ... Ray Kroc has been call

We did not get an output this time because we set the max_iterations paramter while creating an Agent Executor to 3, which means the agent will only hit the google search api 3 times at most and will return the answer 